# Advent of Code 2018

This is my record and commentary of the [2018 Advent of Code](https://adventofcode.com/2018). I'm solving puzzles using Python 3, and when I can, I'm trying to stay up and solve each puzzle at midnight.

Each day, I'm starting with the same template file. The template imports the following modules, which I've found useful for puzzles over the last two years:
* ```copy``` -- Useful for deep copying states in complex search problem. I'll use this if A* search is required at any point.
* ```collections``` -- Helpful if a puzzle calls for a stack or a queue, when a list won't cut it due to running time.
* ```heapq``` -- Priority queues!
* ```itertools``` -- I learned about this during the 2016 competition, and it is extremely useful. It won't be long before I'll use the ```permutations``` or ```combinations``` functions.
* ```functools``` -- Useful for functions on functions. I'm not positive whether I've actually used this on a puzzle in the past, but when I need it, I'll be glad to have it.
* ```math``` -- I'll need square roots eventually.
* ```hashlib``` -- Advent of Code has used MD5 each of the last two years. This is prep for when that happens again.

## Day 1
### Part a -- (192nd, 00:01:54)
Starting from a frequency of 0, walk through a list of frequency changes (puzzle input) and determine the final frequency.

The puzzle input is a series of positive and negative changes in frequency, e.g. ```+1, -2, +3, +1```, with each frequency on its own line. The code is below, but for speed, I wish I had thought to simply call the ```sum()``` function on the puzzle data. When I started, I wasn't sure if something like ```+1``` was a valid string for conversion to an integer. Python allows ```+``` and ```-``` in integer strings.

In [2]:
f = open('input\\input1.txt', 'r')
data = [int(line.strip()) for line in f.readlines()] # List of inputs
f.close()

total = 0
for i in data:
    total += i
print(total)

433


### Part b -- (57th, 00:04:29)
Starting from a frequency of 0, determine the frequency which is reached twice (0 is included as an option). Since this could happen at any position in the list, I'm happy that I'd set up the ```for``` loop for Part (a). Solution below, with comments following.

In [7]:
import sys

f = open('input\\input1.txt', 'r')
data = [int(line.strip()) for line in f.readlines()] # List of inputs
f.close()

total = 0
visited = dict()
visited[0] = True
while True:
    for i in data:
        total += i
        if visited.get(total,False):
            print(total)
            sys.exit(0)
        visited[total] = True

256


SystemExit: 0

c:\users\bo338376\appdata\local\programs\python\python36-32\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


There are a lot of things I did in my solution that works but I hate or consider hack-y.
1. I started out using a set, but for some reason (I'm rusty at these challenges and/or it was midnight and my brain didn't work) I switched to a dictionary with frequencies mapped to ```True``` when they were observed.
2. I blame the ```if``` statement on the same problem (rust/midnight). A better statement would have been ```if total in visited:```, which would have worked for a set or a dictionary. 
3. The ```sys.exit(0)``` line forces the code to quit once a frequency has been duplicated. This is the hackiest thing I did in this solution (and Jupyter hates it, hence the UserWarning above). A better approach would have been to put the puzzle code in a function and return the duplicated value.

The code below is a more optimized approach to both parts. Both parts are in functions, which lets me return the result in Part (b) and avoid using ```sys.exit()```. I've also fixed the other two points I raised above, using a set and making a better ```if``` statement. Finally, instead of nesting the for loop inside a ```while True``` loop, the for loop now iterates over ```cycle(data)```. This function, from the ```itertools``` module, takes any iterable and makes it infinite by starting over when it reaches the end. Since the frequency doesn't duplicate in one pass through the list, this lets us keep repeating the list until the duplication happens.

In [9]:
from itertools import *

f = open('input\\input1.txt', 'r')
data = [int(line.strip()) for line in f.readlines()] # List of inputs
f.close()

def day1a(data):
    return sum(data)

def day1b(data):
    total = 0
    visited = set()
    visited.add(0)
    for i in cycle(data):
        total += i
        if total in visited:
            return total
        visited.add(total)

print('Part a:',day1a(data))
print('Part b:',day1b(data))

Part a: 433
Part b: 256


## Day 2
### Part a -- (58th, 00:02:53)

Given a list of strings, count how many of the strings contain exactly two of any character and exactly three of any character. (A string may qualify as both.) Return the checksum, which is the product of the two totals.

For each string, I construct a dictionary mapping characters to frequencies. After parsing the string, I check the dictionary to see if any of the values are 2 or 3. If so, I increment the counters for when we've seen two of a character or three of a character. 

In [1]:
f = open('input\\input2.txt', 'r')
data = [line.strip() for line in f.readlines()] # List of inputs
f.close()

def day2a(data):
    twos = 0
    threes = 0
    for line in data:
        dct = {}
        sawTwo = False
        sawThree = False
        for ch in line:
            dct[ch] = dct.get(ch,0) + 1
        for key in dct:
            if dct[key] == 2:
                sawTwo = True
            if dct[key] == 3:
                sawThree = True
        if sawTwo:
            twos += 1
        if sawThree:
            threes += 1
    return twos*threes

print(day2a(data))

6944


### Part b (100th, 00:07:45)
The same list of strings contains two strings that are identical except for one character that differs. Identify the identical part of the string, removing the differing character.

My solution doesn't return the string exactly. Instead, it returns the pair of strings. I parsed through the strings manually to determine which character to remove. This was not a great decision, as I accidentally deleted the wrong character on my first attempt. This cost me a minute of time (and 30 places on the leaderboard for this part).

The outer loop iterates over all pairs of strings in the list. The ```combinations()``` function (from ```itertools```) returns all combinations in the set ```data```. The second argument defines the size of the combinations. The inner loop iterates over the strings, looking for indices where the strings are different. If the number of differences ever exceeds 1, then this is not the near-identical pair, so the loop breaks and moves on to the next pair. If after iterating over the string, there was only 1 difference, then the function returns the pair.

In [3]:
from itertools import *

def day2b(data):
    for x,y in combinations(data,2):
        diffs = 0
        for ch in range(len(x)):
            if x[ch] != y[ch]:
                diffs += 1
            if diffs > 1:
                break
        if diffs == 1:
            return x,y

print(day2b(data))

('srijafjzloguvlnvtqmphenbkd', 'srijafjzloguvlnctqmphenbkd')


Overall, I'm happy with my solution to Part (a). I've updated the solution below to use the ```in``` operator to check for 2s and 3s in the dictionary's values. The running time is still $\Theta(n)$, but it makes better use of Python operations.

I made more substantial changes to Part (b) to ensure that the function returns the goal string rather than the matching pair. The ```diffIdx``` variable tracks the index where the differnt character was found. If a second different character is found, the ```found``` field tells us that this is not the matching pair and breaks out of this iteration of the loop. The return statement uses slicing to build the string without the differing character. The solution still runs in $\Theta(n^2)$ time thanks to the ```combinations()``` call. I'm not sure if we can reduce this to linear time or even $\Theta(n$ $lg(n))$ time.

In [4]:
f = open('input\\input2.txt', 'r')
data = [line.strip() for line in f.readlines()] # List of inputs
f.close()

def day2a(data):
    twos = 0
    threes = 0
    for line in data:
        dct = {}
        for ch in line:
            dct[ch] = dct.get(ch,0) + 1
        counts = dct.values()
        if 2 in counts:
            twos += 1
        if 3 in counts:
            threes += 1
    return twos*threes

def day2b(data):
    for x,y in combinations(data,2):
        diffIdx = -1
        found = False
        for ch in range(len(x)):
            if x[ch] != y[ch]:
                if diffIdx >= 0:
                    found = False
                    break
                else:
                    found = True
                    diffIdx = ch
        if found:
            return x[:diffIdx]+x[(diffIdx+1):]

print(day2a(data))
print(day2b(data))


6944
srijafjzloguvlntqmphenbkd
